In [7]:
import http.client, urllib.parse, json
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import html5lib
import os
import sys
from tqdm import tqdm

In [3]:
######## NOS SCRAPE ########
def url_to_html(url):
    """Scrapes the html content from a web page. Takes a URL string as input and returns an html object. """
    # THIS FUNCTION COMES FROM Lab 1 of the course Language as Data 2023, Vrije Universiteit Amsterdam. 
    # Get the html content
    headers = {
    'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/119.0',
    }
    cookies = dict(cookie='kantar_id=1700790955666_87044266; atidvisitor=%7B%22name%22%3A%22atidvisitor%22%2C%22val%22%3A%7B%22vrn%22%3A%22-595271--596579-%22%7D%2C%22options%22%3A%7B%22path%22%3A%22%2F%22%2C%22session%22%3A15724800%2C%22end%22%3A15724800%7D%7D; atuserid=%7B%22name%22%3A%22atuserid%22%2C%22val%22%3A%2249fcf06b-68f6-494b-a5c6-1c2f8b7b60ee%22%2C%22options%22%3A%7B%22end%22%3A%222024-12-25T01%3A55%3A55.697Z%22%2C%22path%22%3A%22%2F%22%7D%7D; CCM_Wrapper_Cache=eyJ2ZXIiOiJ2My4yLjUiLCJqc2giOiIiLCJjaWQiOiJ1dlk0d0JTWDJsRm1pVTJtIiwiY29uaWQiOiJJazlHSCJ9; Cookie_Consent=Fri Nov 24 2023 02:58:03 GMT+0100 (Central European Standard Time); CCM_ID=uvY4wBSX2lFmiU2m; Cookie_Category_Necessary=true; Cookie_Category_Analytics=true')
    
    res = requests.get(url, headers=headers, cookies=cookies)
    #res = requests.get(url + ".pdf", headers={"User-Agent": "XY"})
    html = res.text
    parser_content = BeautifulSoup(html, 'html5lib')
    return parser_content

In [196]:
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait

keyword = "abortus"
search_url = "https://nos.nl/zoeken/?q=" + keyword + "&page=1"
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get(search_url)

time.sleep(10)
driver.find_element("xpath", '/html/body/div[2]/main/div/form/div[2]/div/ul[1]/li[9]/a').click()
#driver.switch_to.frame("#ccm_notification")
#driver.find_element(By.CSS_SELECTOR, "#ccm_close")
#ActionChains(driver).click('<button title="Sluiten" id="ccm_close"></button>').perform()
#driver.find_element("xpath", '/html/body/div[4]//div/div[2]/div/div/button').click()

In [208]:
def get_nos_metadata_with_selenium(keyword, page_range):
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service as ChromeService
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver import ActionChains
    import time as tm
    import re
    
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

    urls = []
    titles = []
    times = []
    search_url = "https://nos.nl/zoeken/?q=" + keyword + "&page=1"
    driver.get(search_url)
    tm.sleep(6)
    i=1
    while i <= page_range:
        tm.sleep(0.1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        links = soup(attrs={'class': 'sc-f75afcb6-4 isiLEZ'})
        result_times = soup(attrs={'class': 'sc-d6d7be46-0 jjSfnY sc-f75afcb6-6 hGGBnM'})
        result_titles = soup(attrs={'class': 'sc-f75afcb6-3 lhteiV'})
        
        for link in links:
            urls.append('https://nos.nl'+link['href'])
            
        for time in result_times:
            article_time = str(time)
            times.append(re.search(r'datetime\=\"(.*?)\"', article_time).group(1))

        for title in result_titles:
            article_title = re.search(r'\<h2 class\=\"sc\-f75afcb6\-3 lhteiV\"\>(.*?)\<\/h2\>', str(title)).group(1)
            titles.append(article_title)
        
        driver.find_element("xpath", '/html/body/div[2]/main/div/form/div[2]/div/ul[1]/li[9]/a').click()
        i += 1
        
    return times, titles, urls

In [8]:
def get_nos_metadata(keyword, page_range):
    '''
    Scrape NOS metadata from the search page since it does not have an official API
    '''
    import time as tm
    urls = []
    titles = []
    times = []
    domain = "https://nos.nl/"
    headers = {
    'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/119.0',
    }
    cookies = dict(cookie='kantar_id=1700790955666_87044266; atidvisitor=%7B%22name%22%3A%22atidvisitor%22%2C%22val%22%3A%7B%22vrn%22%3A%22-595271--596579-%22%7D%2C%22options%22%3A%7B%22path%22%3A%22%2F%22%2C%22session%22%3A15724800%2C%22end%22%3A15724800%7D%7D; atuserid=%7B%22name%22%3A%22atuserid%22%2C%22val%22%3A%2249fcf06b-68f6-494b-a5c6-1c2f8b7b60ee%22%2C%22options%22%3A%7B%22end%22%3A%222024-12-25T01%3A55%3A55.697Z%22%2C%22path%22%3A%22%2F%22%7D%7D; CCM_Wrapper_Cache=eyJ2ZXIiOiJ2My4yLjUiLCJqc2giOiIiLCJjaWQiOiJ1dlk0d0JTWDJsRm1pVTJtIiwiY29uaWQiOiJJazlHSCJ9; Cookie_Consent=Fri Nov 24 2023 02:58:03 GMT+0100 (Central European Standard Time); CCM_ID=uvY4wBSX2lFmiU2m; Cookie_Category_Necessary=true; Cookie_Category_Analytics=true')
    #baggage = dict(baggage = 'sentry-environment=production,sentry-release=production%4015.2.1,sentry-transaction=%2Fzoeken,sentry-public_key=3caf4fd1080f42859504caf5189fa266,sentry-trace_id=c6587f27595b44638850d45e9fc6cdc9,sentry-sample_rate=0')
    
    for i in range(1,page_range+1):
        js_url = "https://atconnect.npo.nl/hit.xiti?s=595271&idclient=49fcf06b-68f6-494b-a5c6-1c2f8b7b60ee&ts=1700791633386&vtag=5.29.1&ptag=js&r=2560x1440x24x24&re=2490x1328&hl=3x7x13&lng=en-US&p=site.search.pagination." + str(i) + ".open&click=A&s2=1&pclick=search.abortus.open&s2click=1"
        requests.post(js_url, headers=headers,cookies=cookies)
        search_url = "https://nos.nl/zoeken/?q=" + keyword + "&page=" + str(i)
        requests.post("https://o314617.ingest.sentry.io/api/5686804/envelope/?sentry_key=3caf4fd1080f42859504caf5189fa266&sentry_version=7&sentry_client=sentry.javascript.nextjs%2F7.39.0", headers=headers)
        requests.post("https://o314617.ingest.sentry.io/api/5686804/envelope/?sentry_key=3caf4fd1080f42859504caf5189fa266&sentry_version=7&sentry_client=sentry.javascript.nextjs%2F7.39.0", headers=headers)
        parser_content= url_to_html(search_url)
        result_url = parser_content.find_all("a", {"class":"sc-f75afcb6-4 isiLEZ"})
        result_times = parser_content.find_all("div",{"class":"sc-d6d7be46-0 jjSfnY sc-f75afcb6-6 hGGBnM"})
        result_titles = parser_content.find_all("h2",{"class":"sc-f75afcb6-3 lhteiV"})
        for j, url in enumerate(result_url):
            article_url = domain + url["href"]
            urls.append(article_url)
        for m, title in enumerate(result_titles):
            article_title = re.search(r'\<h2 class\=\"sc\-f75afcb6\-3 lhteiV\"\>(.*?)\<\/h2\>', str(title)).group(1)
            titles.append(article_title)
        for k,time in enumerate(result_times):
            article_time = re.search(r'time datetime\=\"(.*?)\"\>', str(time)).group(1)
            times.append(article_time)
            
        tm.sleep(40)
    return times, titles, urls

nos_metadata2 = get_nos_metadata(keyword = "abortus", page_range = 2) # scraping some extra links as there may be invalid video links

In [209]:
nos_metadata = get_nos_metadata_with_selenium(keyword = "abortus", page_range = 40)

In [210]:
def remove_nos_dups_and_vids(metadata):
    '''
    remove the duplicates and
    NOS search results sometimes includes video links
    example: https://nos.nl/nieuwsuur/video/2496544-cu-lid-annelijn-door-wie-wil-je-eigenlijk-aardig-gevonden-worden
    This function removes such links
    '''
    df = pd.DataFrame(metadata).T.rename(columns={0:'time', 1:'title', 2:'url'})
    df = df.drop_duplicates(subset='url', keep="last")
    video_pattern = "\/video\/"
    filter = df['url'].str.contains(video_pattern)
    cleaned_df = df[~filter].head(600)
    
    return cleaned_df

cleaned_nos_metadata = remove_nos_dups_and_vids(nos_metadata)

In [216]:
cleaned_nos_metadata.head(20)["url"]

0     https://nos.nl/artikel/2497109-conservatieve-s...
2     https://nos.nl/artikel/2495085-podcast-de-dag-...
3     https://nos.nl/artikel/2494020-met-vrouwenrech...
4     https://nos.nl/artikel/2493865-aantal-abortuss...
6     https://nos.nl/artikel/2489520-hoogste-hof-mex...
7     https://nos.nl/artikel/2485970-regels-referend...
8     https://nos.nl/artikel/2485959-referendum-ohio...
10    https://nos.nl/artikel/2486982-stijgende-sterf...
11    https://nos.nl/nieuwsuur/artikel/2497513-nieuw...
12    https://nos.nl/nieuwsuur/artikel/2496542-chris...
13    https://nos.nl/artikel/2497388-poolse-oppositi...
14    https://nos.nl/artikel/2494631-belangrijkste-m...
15    https://nos.nl/artikel/2495031-nog-geen-nieuwe...
16    https://nos.nl/artikel/2498505-uiterst-rechtse...
17    https://nos.nl/artikel/2492262-sgp-kwetsbaar-l...
18    https://nos.nl/liveblog/2497872-beeld-in-peili...
19    https://nos.nl/artikel/2482705-amerikaanse-vro...
20    https://nos.nl/artikel/2497245-vvd-en-gl-p

In [ ]:
soup = BeautifulSoup(requests.get(url).text,"html5lib")
section_container = soup.select("p",{"class":"sc-f9df6382-7 cMuisv"}) # NOS section class
section = re.search(r'cMuisv\"\>(.*?)\<\/p\>', str(section_container)).group(1)
text = ""
for item in soup.select("p",{"class":"sc-6d77a1d1-0 chzewu"}): # NOS body text class
    text += item.text.strip()